In [13]:
from gtts import gTTS
import pydub
from pydub import AudioSegment, effects
from io import BytesIO
from moviepy.editor import AudioFileClip, ImageClip, CompositeVideoClip
from google.cloud import speech_v1p1beta1 as speech
from google.cloud.speech_v1p1beta1 import types
import re
import tempfile

In [14]:
song_lyrics = """
Gather 'round folks, let me tell you a tale,
Of a fantasy league where challenges never fail,
For three long years, the championship's theme,
Freddy was on top, living the dream.

Kel was the runner-up, again and again,
Every year, it seemed he just couldn't win,
The first season was a bit of a trip,
With Freddy in 8th, he still took the 'ship

Oh, Freddy and Kel, rivals for days,
Battling it out in so many ways,
Year after year, the story's the same,
Who'll be the champ of the fantasy game

The first year’s vote, was quite the scene,
Should we expand to an 8-man routine?
It was tied four to four, a decision to keep,
Freddy made the call, and took the leap.

Kel had a strong squad, year number two,
But Freddy's team, out of the blue,
Surpassed them all, took the gold once more,
Leaving Kel to ponder, what’s in store

Oh, Freddy and Kel, rivals for days,
Battling it out in so many ways,
Year after year, the story's the same,
Who’ll be the champ of the fantasy game

Year three came around, Kel's team was on fire,
Only two losses, aspirations higher,
But come playoff time, the story's old,
Freddy took the 'ship, Kel left in the cold.

Now year four is here, the drama unfolds,
Six weeks in, and the tale that’s told,
Both Freddy and Kel, at the top they stand,
Who'll reign supreme, in fantasy land

Kel's blockbuster trade, oh what a sensation,
Got Justin Jefferson, sparking elation,
Will this be the year, he breaks the trend
Or will Freddy’s reign, simply extend

Oh, Freddy and Kel, rivals for days,
Battling it out in so many ways,
Year after year, the story's the same,
Who’ll be the champ of the fantasy game

The season’s not done, the story’s still raw,
Who'll be the victor, who will draw?
Tune in next time, for the end of this tale,
In the fantasy league, where legends prevail.
"""

In [15]:
tts = gTTS(text=song_lyrics, lang='en')
with BytesIO() as buffer:
    tts.write_to_fp(buffer)
    buffer.seek(0)
    audio_segment = AudioSegment.from_file(buffer)

audio_segment.export('fantasy_reading.mp3', format='mp3')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [32]:
# Change playback speed (speed up by 1.5x for example)
fast_audio = audio_segment.speedup(playback_speed=1.25)

fast_audio.export('fantasy_reading_fast.mp3', format='mp3')

<_io.BufferedRandom name='fantasy_reading_fast.mp3'>

In [33]:
speech_audio = pydub.AudioSegment.from_mp3("fantasy_reading_fast.mp3")
background_music = pydub.AudioSegment.from_mp3("nothing.mp3")
combined_audio = speech_audio.overlay(background_music-10, loop=True)
combined_audio.export("fantasy_rap.mp3", format="mp3")

<_io.BufferedRandom name='fantasy_rap.mp3'>

In [34]:
# Transcribe the audio to generate the SRT file
client = speech.SpeechClient()
with open("fantasy_rap.mp3", "rb") as audio_file:
    content = audio_file.read()
audio = speech.RecognitionAudio(content=content)
config = types.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.MP3,
    sample_rate_hertz=44100,
    language_code="en-US",
    enable_automatic_punctuation=True,
    enable_word_time_offsets=True
)

response = client.recognize(config=config, audio=audio)

In [35]:
response

results {
  alternatives {
    transcript: "Gather round. Folks, let me tell you A Tale of a fantasy league where challenges never fail for three long years. The Championships being Freddy was on top living the dream tell was the runner-up again, and again every year, it seems he just couldn\'t win the first season with a bit of a trip. With Freddy at 8, he still took the ship o Freddy and Kel Rivals for days, battling it out in so many ways year after year, the story\'s the same school be the champ of the fantasy game. The first Year\'s vote was quite the scene. Should we expand to an eight-man routine. It was tied for 2 for my decision to keep Freddy made the call and took the leaf. Kill had a strong Squad, your number to but Freddy\'s team out of the blue surpassed them all took the gold was more leaving Carroll to ponder what\'s in store 00 Freddy and tell vitals for days. Battling it out in so many ways year after year, the story\'s the same school be the judge."
    confidence: 0

In [36]:
from moviepy.editor import TextClip, concatenate_videoclips

audio_clip = AudioFileClip("fantasy_rap.mp3")
image_file = "stained_glass_football.png"
image_clip = ImageClip(image_file, duration=audio_clip.duration)

# Generate clips for each text segment and overlay them on the image
clips = []
prev_end_time = 0
for result in response.results:
    chunks = re.split(r'(?<=[.!?]) +', result.alternatives[0].transcript)
    for i, word in enumerate(result.alternatives[0].words):
        for chunk in chunks:
            if chunk.endswith(word.word) and chunk in result.alternatives[0].transcript:
                start_seconds = result.alternatives[0].words[i - len(chunk.split()) + 1].start_time.total_seconds()
                end_seconds = word.end_time.total_seconds()
                segment_duration = end_seconds - start_seconds

                # Create a segment of the video with the text overlay
                txt_clip = (TextClip(chunk, fontsize=24, color='white')
                            .set_pos('bottom')
                            .set_duration(segment_duration)
                            .set_start(start_seconds))
                segment = CompositeVideoClip([image_clip.subclip(prev_end_time, start_seconds + segment_duration), txt_clip])
                clips.append(segment)
                prev_end_time = start_seconds + segment_duration


# Concatenate all the video segments to form the final video
final_video = concatenate_videoclips(clips)
final_video = final_video.set_audio(audio_clip)
final_video.write_videofile("fantasy_vid.mp4", codec="libx264", audio_codec="aac", fps=24)